# Operações Avançadas no Numpy

Vale lembrar que sempre que iniciamos um novo arquivo jupyter e o kernel estiver zerado (nada tiver sido compilado) necessitamos compilar os módulos que necessitamos importar:

In [2]:
import numpy as np

## Copiando Arrays

```{note}
Simplesmente usar "=" não faz uma cópia, mas, assim como com listas, você terá vários nomes apontando para o mesmo objeto ndarray.
```

Portanto, precisamos entender se dois arrays, `A` e `B`, apontam para:
* o mesmo array, incluindo forma e espaço de dados/memória
* o mesmo espaço de dados/memória, mas talvez formas diferentes (uma _view_)
* uma cópia separada dos dados (ou seja, armazenada completamente separada na memória)

Todos esses casos são possíveis:
* `B = A`

  isso é _atribuição_. Nenhuma cópia é feita. `A` e `B` apontam para os mesmos dados na memória e compartilham a mesma forma, etc. Eles são apenas dois rótulos diferentes para o mesmo objeto na memória.

* `B = A[:]`

  isso é uma _view_ ou _cópia rasa_. As informações de forma de A e B são armazenadas independentemente, mas ambos apontam para o mesmo local de memória para os dados.

* `B = A.copy()`

  isso é uma _cópia profunda_. Um objeto completamente separado será criado na memória, com um local completamente separado na memória.

Vamos ver exemplos:


In [53]:
a = np.arange(10)
print(a)

[0 1 2 3 4 5 6 7 8 9]


Aqui está a atribuição — podemos simplesmente usar o operador `is` para testar a igualdade.

In [54]:
b = a
b is a

True

Como `b` e `a` são o mesmo, alterações na forma de um são refletidas no outro — nenhuma cópia é feita.

In [55]:
b.shape = (2, 5)
print(b)
a.shape

[[0 1 2 3 4]
 [5 6 7 8 9]]


(2, 5)

In [56]:
b is a

True

In [57]:
print(a)

[[0 1 2 3 4]
 [5 6 7 8 9]]


Uma cópia rasa cria uma nova *view* do array — os _dados_ são os mesmos, mas as _propriedades_ do array podem ser diferentes.

In [58]:
a = np.arange(12)
c = a[:]
a.shape = (3,4)

print(a)
print(c)

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[ 0  1  2  3  4  5  6  7  8  9 10 11]


Como os dados subjacentes estão na mesma memória, alterar um elemento de um é refletido no outro.

In [59]:
c[1] = -1
print(a)

[[ 0 -1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


Até mesmo fatiamentos em um array são apenas views, ainda apontando para a mesma memória.

In [60]:
d = c[3:8]
print(d)

[3 4 5 6 7]


In [61]:
d[:] = 0 

In [62]:
print(a)
print(c)
print(d)

[[ 0 -1  2  0]
 [ 0  0  0  0]
 [ 8  9 10 11]]
[ 0 -1  2  0  0  0  0  0  8  9 10 11]
[0 0 0 0 0]


Existem várias maneiras de verificar se dois arrays são iguais, são views, possuem seus próprios dados, etc.

In [63]:
print(c is a)
print(c.base is a)
#print(c.flags.owndata)
#print(a.flags.owndata)

False
True


Para fazer uma cópia dos dados do array com a qual você possa trabalhar independentemente do original, você precisa de uma _cópia profunda_.

In [64]:
d = a.copy()
d[:,:] = 0.0

print(a)
print(d)

[[ 0 -1  2  0]
 [ 0  0  0  0]
 [ 8  9 10 11]]
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


## Indexação booleana (também conhecida como mascaramento)

Existem várias maneiras interessantes de indexar arrays para acessar apenas os elementos que atendem a uma determinada condição.

In [4]:
a = np.arange(12).reshape(3,4)
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

Aqui, definimos todos os elementos do array que são > 4 para zero.

In [5]:
a > 4

array([[False, False, False, False],
       [False,  True,  True,  True],
       [ True,  True,  True,  True]])

In [6]:
a[a > 4]

array([ 5,  6,  7,  8,  9, 10, 11])

In [7]:
a[a > 4] = 0
a

array([[0, 1, 2, 3],
       [4, 0, 0, 0],
       [0, 0, 0, 0]])

E agora, todos os zeros para -1.

In [8]:
a[a == 0] = -1
a

array([[-1,  1,  2,  3],
       [ 4, -1, -1, -1],
       [-1, -1, -1, -1]])

In [9]:
a == -1

array([[ True, False, False, False],
       [False,  True,  True,  True],
       [ True,  True,  True,  True]])

Vamos considerar outro exemplo, envolvendo dois arrays distintos

In [ ]:
a = np.arange(12)
b = np.arange(12)*123

In [ ]:
b

array([   0,  123,  246,  369,  492,  615,  738,  861,  984, 1107, 1230,
       1353])

In [ ]:
a>6

array([False, False, False, False, False, False, False,  True,  True,
        True,  True,  True])

In [ ]:
b[a>6]

array([ 861,  984, 1107, 1230, 1353])

Se tivermos 2 testes, precisamos usar `logical_and()` ou `logical_or()`.

In [75]:
a = np.arange(12).reshape(3,4)
a[np.logical_and(a > 3, a <= 9)] = 0.0
a

array([[ 0,  1,  2,  3],
       [ 0,  0,  0,  0],
       [ 0,  0, 10, 11]])

## Evitando loops

De maneira geral, você quer evitar loops sobre os elementos de um array.

Aqui, vamos criar coordenadas 1D de x e y e depois tentar preencher um array maior.

In [11]:
M = 3200
N = 6400
xmin = ymin = 0.0
xmax = ymax = 1.0

x = np.linspace(xmin, xmax, M, endpoint=False)
y = np.linspace(ymin, ymax, N, endpoint=False)

print(x.shape)
print(y.shape)

(3200,)
(6400,)


Vamos medir o tempo do nosso código.

In [12]:
import time

In [13]:
t0 = time.time()

g = np.zeros((M, N))

# Índices, que ideia terrível
for i in range(M):
    for j in range(N):
        g[i,j] = np.sin(2.0*np.pi*x[i]*y[j])
        
t1 = time.time()
print("tempo decorrido: {} s".format(t1-t0))

tempo decorrido: 16.666316509246826 s


Agora, vamos fazer isso usando toda a sintaxe de arrays. Primeiro, vamos estender nossos arrays de coordenadas 1D para 2D. O Numpy tem uma função para isso (`meshgrid()`).

Vamos ver como o `meshgrid()` funciona primeiro.


In [16]:
x2d, y2d = np.meshgrid([0,1,2,3], [10,20,30], indexing="ij")
x2d

array([[0, 0, 0],
       [1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]])

In [17]:
y2d

array([[10, 20, 30],
       [10, 20, 30],
       [10, 20, 30],
       [10, 20, 30]])

Vemos que isso cria 2 arrays bidimensionais, um com os valores de x variando ao longo das linhas e outro com os valores de y variando ao longo das colunas. Isso significa que podemos indexar todos os pontos (`x[i]`, `y[j]`) através do par `x2d`, `y2d`.

Agora, vamos fazer o mesmo exemplo usando este método.

In [14]:
t0 = time.time()
x2d, y2d = np.meshgrid(x, y, indexing="ij")
g2 = np.sin(2.0*np.pi*x2d*y2d)
t1 = time.time()
print("tempo decorrido: {} s".format(t1-t0))

tempo decorrido: 0.21164917945861816 s


No meu laptop, isso é cerca de:Uma verificação final para garantir que eles forneçam a mesma resposta.


In [18]:
print(np.max(np.abs(g2-g)))

0.0


### Exemplo: Diferenciação Numérica

Agora, queremos construir uma derivada,
$$
\frac{d f}{dx}
$$


In [19]:
x = np.linspace(0, 2*np.pi, 25)
f = np.sin(x)

Queremos fazer isso sem loops — usaremos views em arrays deslocados entre si. Lembre-se de cálculo que uma derivada é aproximadamente:
$$
\frac{df}{dx} = \frac{f(x+h) - f(x)}{h}
$$
Aqui, tomaremos $h$ como um único elemento adjacente.

In [84]:
f

array([ 0.00000000e+00,  2.58819045e-01,  5.00000000e-01,  7.07106781e-01,
        8.66025404e-01,  9.65925826e-01,  1.00000000e+00,  9.65925826e-01,
        8.66025404e-01,  7.07106781e-01,  5.00000000e-01,  2.58819045e-01,
        1.22464680e-16, -2.58819045e-01, -5.00000000e-01, -7.07106781e-01,
       -8.66025404e-01, -9.65925826e-01, -1.00000000e+00, -9.65925826e-01,
       -8.66025404e-01, -7.07106781e-01, -5.00000000e-01, -2.58819045e-01,
       -2.44929360e-16])

In [85]:
f[1:]

array([ 2.58819045e-01,  5.00000000e-01,  7.07106781e-01,  8.66025404e-01,
        9.65925826e-01,  1.00000000e+00,  9.65925826e-01,  8.66025404e-01,
        7.07106781e-01,  5.00000000e-01,  2.58819045e-01,  1.22464680e-16,
       -2.58819045e-01, -5.00000000e-01, -7.07106781e-01, -8.66025404e-01,
       -9.65925826e-01, -1.00000000e+00, -9.65925826e-01, -8.66025404e-01,
       -7.07106781e-01, -5.00000000e-01, -2.58819045e-01, -2.44929360e-16])

In [86]:
f[:-1]

array([ 0.00000000e+00,  2.58819045e-01,  5.00000000e-01,  7.07106781e-01,
        8.66025404e-01,  9.65925826e-01,  1.00000000e+00,  9.65925826e-01,
        8.66025404e-01,  7.07106781e-01,  5.00000000e-01,  2.58819045e-01,
        1.22464680e-16, -2.58819045e-01, -5.00000000e-01, -7.07106781e-01,
       -8.66025404e-01, -9.65925826e-01, -1.00000000e+00, -9.65925826e-01,
       -8.66025404e-01, -7.07106781e-01, -5.00000000e-01, -2.58819045e-01])

In [21]:
print(len(f[:-1]))
len(f[1:])

24


24

In [90]:
dx = x[1] - x[0]
dfdx = (f[1:] - f[:-1])/dx

In [103]:
dfdx

array([ 0.98861593,  0.92124339,  0.79108963,  0.60702442,  0.38159151,
        0.13015376, -0.13015376, -0.38159151, -0.60702442, -0.79108963,
       -0.92124339, -0.98861593, -0.98861593, -0.92124339, -0.79108963,
       -0.60702442, -0.38159151, -0.13015376,  0.13015376,  0.38159151,
        0.60702442,  0.79108963,  0.92124339,  0.98861593])